In [39]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json

In [71]:
distances = pd.read_excel('data/distances.xlsx', index_col=0)
events18 = pd.read_csv('data/events18.csv',sep='\t')
events19 = pd.read_csv('data/events19.csv',sep='\t')
events20 = pd.read_csv('data/events20.csv',sep='\t')
events21 = pd.read_csv('data/events21.csv',sep='\t')
events = pd.concat([events18,events19,events20,events21]).reset_index(drop=True)
sallesDeCours = pd.read_csv('data/SallesDeCours.csv',sep=';')
students18 = json.load(open('data/students18.json'))
students19 = json.load(open('data/students19.json'))
students20 = json.load(open('data/students20.json'))
students21 = json.load(open('data/students21.json'))
students = {**students18,**students19,**students20,**students21}
events.head()

,Event,Start,End,Code cours
0,10178-07,18-10-2021 10:00:00,18-10-2021 12:00:00,TRAD0110-B-a
1,10180-07,18-10-2021 16:00:00,18-10-2021 18:00:00,TRAD0096-A-a
2,1024-07,18-10-2021 08:30:00,18-10-2021 10:30:00,PHAC0341-A-b
3,10258-07,18-10-2021 14:00:00,18-10-2021 16:00:00,VETE0432-A-d
4,10398-07,18-10-2021 15:45:00,18-10-2021 17:45:00,CHIM0310-B-a
